# Interactively inspect Target Pixel Files (TPF) and lightcurves

One of the most common questions we get at the Kepler/K2 Guest Observer Office is "I see a noise blip in my K2 lightcurve.  Is it real or instrumental?"  Answering the question usually entails ticking through our (long, incomplete) mental list of conceivable, sometimes exotic instrumental artifacts.  Space-craft induced periodic motion.  Rolling band.  Cosmic Rays.  Collateral cosmic rays.  Smear.  Sudden pixel sensitivity dropouts.  Fine-guidance-sensor cross talk.  Regular cross talk. [Sweater fuzzy](https://twitter.com/gully_/status/941707161058586624).

In this tutorial we introduce and explain the `tpf.interact()` tool that accomplishes the goal of interactively inspecting the TPFs and lightcurve simultaneously.

### Using `.interact()`

Using interact should be as simple as downloading a Kepler Target Pixel File (TPF) and running the method `.interact()`.  This method can only be run in a Jupyter Notebook at the moment, and is limited to lightcurves with fewer than 30000 cadences.

In [1]:
from lightkurve import KeplerTargetPixelFile
tpf = KeplerTargetPixelFile.from_archive(249928278, campaign=15) # WASP 104 for example

INFO: Found cached file ./mastDownload/K2/ktwo249928278-c15_lc/ktwo249928278-c15_lpd-targ.fits.gz with expected size 6996829. [astroquery.query]


The default lightcurves comes from the default "pipeline" mask, typically only a few pixels centered on the target photocenter, depending on factors like the target brightness and its expected PSF size.  

In [2]:
tpf.interact()

Loading BokehJS ...

A Jupyter Widget

A Jupyter Widget

You can move the large bottom left slider to change the location of the verical red bar, which indicates which cadence is being shown in the TPF postage stamp image.  You can also hit the play button to automatically play-though all the cadences.  The left-and-right arrow keys on your keyboard will tick through the cadences one-by-one, if the widget slider is active (i.e. it's the most recent thing you clicked on).  The slider beneath the TPF postage stamp image controls the screen stretch, which defaults to logarithmic scaling initialized to 1% and 95% lower and upper limits respectively.

You can move your cursor over individual data points to show hover-over tool-tips indicating additional information about that datum.  Currently the tool tips list the cadence, time, flux, and quality flags.  The tools on the right hand side of the plots enable zooming, panning, and toggling-off tool-tips.

### Passing in a preprocessed lightcurve

You may wish to visualize a custom-processed lightcurve adjacent to its TPF postage stamp representation.  You can pass-in a lightcurve as an optional keyword argument like so:

In [3]:
import numpy as np

In [4]:
def my_custom_lc_cleaner(tpf):
    '''Returns a custom lightcurve that uses local background subtraction'''
    
    # Define the background as the outermost ring of non-NaN pixels
    aper_mask = tpf.pipeline_mask * False
    aper_mask[1:-1, 1:-1] = True
    back_mask = ~aper_mask & ~np.all(np.isnan(tpf.flux), axis=0)
    
    # Estimate the background level by the outer ring of pixels
    lc_aper = tpf.to_lightcurve(aperture_mask=aper_mask) / aper_mask.sum()
    lc_back = tpf.to_lightcurve(aperture_mask=back_mask) / back_mask.sum()
    
    lc_bg_sub = lc_aper - lc_back.flux
    
    # Remove outliers
    lc_cln= lc_bg_sub.remove_outliers()
    
    return lc_cln

In [5]:
lc_cln = my_custom_lc_cleaner(tpf)

In [6]:
tpf.interact(lc=lc_cln)

Loading BokehJS ...

A Jupyter Widget

A Jupyter Widget

Your clean lightcurve may have a different number of cadences than your target pixel file, for example if you dropped outliers.  The TPF will show all of the available cadences by default, with missing spaces in the lightcurve.

We hope you enjoy the `.interact()` tool!  Requests for extensions or more features are welcomed on our [GitHub Issues](https://github.com/KeplerGO/lightkurve/issues).